In [2]:
pip install cv2

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for cv2


In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import zipfile

# Unzip the Dataset
zip_file_path = "Dataset.zip"  # Replace with the actual path to your zip file
extract_dir = "."  # Extract to the current directory

try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Successfully extracted '{zip_file_path}' to '{extract_dir}'")
except zipfile.BadZipFile:
    print(f"Error: '{zip_file_path}' is not a valid zip file.")
except FileNotFoundError:
    print(f"Error: The file '{zip_file_path}' was not found.")
except Exception as e:
    print(f"An error occurred during extraction: {e}")


# Data Loading and Preprocessing
def load_and_preprocess_data(data_dir):
    images = []
    labels = []
    for folder in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, folder)
        if os.path.isdir(folder_path):
            for image_file in os.listdir(folder_path):
                image_path = os.path.join(folder_path, image_file)
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
                if image is not None:
                    image = cv2.resize(image, (50, 50))  # Resize for consistency
                    images.append(image)
                    labels.append(folder)
    return np.array(images), np.array(labels)

# Main Data Loading
train_dir = 'Dataset/train'  # Changed to 'Dataset/train'
val_dir = 'Dataset/val'    # Changed to 'Dataset/val'
test_dir = 'Dataset/test'   # Changed to 'Dataset/test'

train_images, train_labels = load_and_preprocess_data(train_dir)
val_images, val_labels = load_and_preprocess_data(val_dir)
test_images, test_labels = load_and_preprocess_data(test_dir)

# Expand dimensions to (None, 50, 50, 1)
train_images = np.expand_dims(train_images, axis=-1)
val_images = np.expand_dims(val_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)


# Label Encoding
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# One-hot encode the labels
train_labels_categorical = to_categorical(train_labels_encoded)
val_labels_categorical = to_categorical(val_labels_encoded)
test_labels_categorical = to_categorical(test_labels_encoded)


# Normalize pixel values to be between 0 and 1
train_images = train_images.astype('float32') / 255.0
val_images = val_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

print("Train images shape:", train_images.shape)
print("Train labels shape:", train_labels_categorical.shape)
print("Test images shape:", test_images.shape)
print("Test labels shape:", test_labels_categorical.shape)

Successfully extracted 'Dataset.zip' to '.'
Train images shape: (6895, 50, 50, 1)
Train labels shape: (6895, 10)
Test images shape: (985, 50, 50, 1)
Test labels shape: (985, 10)


In [5]:
# Model Architecture (Convolutional Neural Network)
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')  # Output layer
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 48, 48, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 24, 24, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 22, 22, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 11, 11, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 9, 9, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 356,234 (1.36 MB)

 Trainable params: 356,234 (1.36 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Model Training
history = model.fit(
    train_images, train_labels_categorical,
    validation_data=(val_images, val_labels_categorical),
    epochs=20,  # You can adjust the number of epochs
    batch_size=32
)

# Model Evaluation
test_loss, test_accuracy = model.evaluate(test_images, test_labels_categorical, verbose=0)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.4832 - loss: 1.4539 - val_accuracy: 0.9304 - val_loss: 0.2175
Epoch 2/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9106 - loss: 0.2860 - val_accuracy: 0.9604 - val_loss: 0.1365
Epoch 3/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9514 - loss: 0.1490 - val_accuracy: 0.9705 - val_loss: 0.1121
Epoch 4/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9644 - loss: 0.1172 - val_accuracy: 0.9705 - val_loss: 0.1113
Epoch 5/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9750 - loss: 0.0839 - val_accuracy: 0.9731 - val_loss: 0.0981
Epoch 6/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9782 - loss: 0.0762 - val_accuracy: 0.9741 - val_loss: 0.1001
Epoch 7/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9825 - loss: 0.0658 - val_accuracy: 0.9787 - val_loss: 0.0782
Epoch 8/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9831 - loss: 0.0498 - val_accuracy: 

In [7]:
def predict_roman_numeral(image_path, model, label_encoder):
    """
    Predicts the Roman numeral in a given image.

    Args:
        image_path (str): Path to the image file.
        model: Trained Keras model.
        label_encoder: LabelEncoder used for encoding.

    Returns:
        str: Predicted Roman numeral, or None if prediction fails.
    """
    try:
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"Error: Could not read image at {image_path}")
            return None
        image = cv2.resize(image, (50, 50))
        image = np.expand_dims(image, axis=0)  # Add batch dimension
        image = np.expand_dims(image, axis=-1) # Add channel dimension
        image = image.astype('float32') / 255.0

        prediction = model.predict(image)
        predicted_class_index = np.argmax(prediction)
        predicted_roman_numeral = label_encoder.inverse_transform([predicted_class_index])[0]
        return predicted_roman_numeral
    except Exception as e:
        print(f"An error occurred during prediction: {e}")
        return None

In [15]:
# Example Usage:
test_image_path = "/content/sample3.png"  # Replace with a valid test image path
predicted_numeral = predict_roman_numeral(test_image_path, model, label_encoder)

if predicted_numeral:
    print(f"Predicted Roman Numeral: {predicted_numeral}")
else:
    print("Prediction failed.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Predicted Roman Numeral: vi


In [16]:
# Save the trained model
model.save('roman_numeral_predictor.keras')
print("Model saved as 'roman_numeral_predictor.keras'")

Model saved as 'roman_numeral_predictor.keras'


In [17]:
print('done')

done
